<a href="https://colab.research.google.com/github/Dr-Doofensmirtz/my_projects/blob/master/data_science_bowl_2018/1_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# getting data

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


downloading data from kaggle

In [11]:
import os
try:
  os.environ['KAGGLE_CONFIG_DIR'] = "drive/MyDrive/kaggle/"
except:
  print('path not found')

In [10]:
!kaggle competitions download -c data-science-bowl-2018

stage1_train.zip: Skipping, found more recently modified local copy (use --force to force download)
stage2_test_final.zip: Skipping, found more recently modified local copy (use --force to force download)
stage1_solution.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
stage1_test.zip: Skipping, found more recently modified local copy (use --force to force download)
stage2_sample_submission_final.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
stage1_sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
stage1_train_labels.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


extracting training data

In [35]:
import zipfile
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import shutil
import glob

from skimage.io import *
from skimage.transform import resize
from sklearn import model_selection

from joblib import Parallel, delayed

from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils
from torch import nn

import albumentations as A
from albumentations.pytorch import ToTensor

In [14]:
with zipfile.ZipFile("stage1_train.zip", "r") as file:
  file.extractall("train_data/")

In [52]:
def get_mask(mask_dir, IMG_HEIGHT, IMG_WIDTH):   
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.uint8)
    
    for mask_f in next(os.walk(mask_dir))[2]:
        mask_ = imread(mask_dir)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT,IMG_WIDTH), mode='constant', preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    
    os.mkdir(s_path)
    imsave(s_path+ img +".jpg", mask)
    shutil.rmtree(m_path)

In [29]:
def get_folds(df):

  df['kfold'] = -1
  df = df.sample(frac=1).reset_index(drop=True)
  kf = model_selection.KFold(n_splits=5)

  for fold_, (_, x) in enumerate(kf.split(df)):
      for xs in x:
          df.loc[xs, "kfold"] = fold_

  return df
  # print(df.head())
  # print(df.tail())
  # df.to_csv("folds.csv", index=False)

In [55]:
img_dir = glob.glob("train_data/*/images/*.png")
mask_dir = glob.glob("train_data/*/masks/")

In [54]:
mask_dir[1]

'train_data/bbfc4aab5645637680fa0ef00925eea733b93099f1944c0aea09b78af1d4eef2/mask/'

In [26]:
data = {"img_": img_dir, "mask_": mask_dir}
df = pd.DataFrame(data=data)

In [27]:
df.head()

,img_,mask_
0,train_data/c0c4a829c8d33d16a02f5dc0411597329f4...,train_data/c0c4a829c8d33d16a02f5dc0411597329f4...
1,train_data/bbfc4aab5645637680fa0ef00925eea733b...,train_data/bbfc4aab5645637680fa0ef00925eea733b...
2,train_data/cf26c41245febfe67c2a1682cc4ee8752ee...,train_data/cf26c41245febfe67c2a1682cc4ee8752ee...
3,train_data/76a372bfd3fad3ea30cb163b560e52607a8...,train_data/76a372bfd3fad3ea30cb163b560e52607a8...
4,train_data/0e21d7b3eea8cdbbed60d51d72f4f8c1974...,train_data/0e21d7b3eea8cdbbed60d51d72f4f8c1974...


In [32]:
df_fold = get_folds(df)

In [34]:
df_fold.tail()

,img_,mask_,kfold
665,train_data/8f27ebc74164eddfe989a98a754dcf5a9c8...,train_data/8f27ebc74164eddfe989a98a754dcf5a9c8...,4
666,train_data/602f267432e7a573e1092f1cf48135c82d0...,train_data/602f267432e7a573e1092f1cf48135c82d0...,4
667,train_data/76a372bfd3fad3ea30cb163b560e52607a8...,train_data/76a372bfd3fad3ea30cb163b560e52607a8...,4
668,train_data/ed5be4b63e9506ad64660dd92a098ffcc03...,train_data/ed5be4b63e9506ad64660dd92a098ffcc03...,4
669,train_data/c15c652c08153fb781a5349123ab8f80bb2...,train_data/c15c652c08153fb781a5349123ab8f80bb2...,4


In [43]:
def get_train_transform():
   return A.Compose(
       [
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        A.HorizontalFlip(p=0.25),
        A.VerticalFlip(p=0.25),
        ToTensor()
        ])

In [ ]:
class dataSet(Dataset):
  def __init__(self, df, fold=0, train=True, transforms= None):
    self.transforms = transforms

    if train:
      self.df = df[df.kfold != fold].reset_index(drop=True)
    else:
      self.df = df[df.kfold == fold].reset_index(drop=True)

  def __getitem__(self, idx):
    img_path = df.loc[idx, "img_"]
    mask_path = df.loc[idx, "mask_"]

    mask = get_mask(mask_path)
    img = imread(img_path)
    mask = imread(mask_path)

    if self.transforms:


In [50]:
x = imread(df.loc[56,"img_"])

In [51]:
x.shape

(256, 320, 4)